In [1]:
import sys
import json
import random
from datetime import datetime
from datetime import timedelta

from function.alphas_function import *
from function.db_function import *

from config import *


In [ ]:


k = 0
max_submit = 2

user = MyUser('user.db')
print(user.get_column('email'))
author = user.get_column('author')

hidden = 'no'
submit_fitness = '5' # quan trong
print(submit_fitness)

combo_dess = [
    "This expression evaluates to 1 for every Alpha every day. It will weight each selected Alpha equally every day.",
    "This expression weights each Alpha by its information ratio from the previous 250 trading days (~1 year).",
    "This expression weights each Alpha by the maximum correlation with all other selected Alphas over a 2 year period, with more correlated Alphas receiving less weight."
]

# combo_dess = ["This expression evaluates to 1 for every Alpha every day. It will weight each selected Alpha equally every day."]

if hidden == 'no':
    submit_prod_corr = 0.7
    submit_self_corr = 0.7
else:
    submit_prod_corr = 1
    submit_self_corr = 0.7


api2 = Infor()

api2.get_base_payment()

last_update = datetime.now() - timedelta(weeks = 4)
date_start = last_update.strftime('%Y-%m-%d')
last_update = last_update.strftime('%Y-%m-%dT00:00:00-00:00')

In [ ]:

while True:
    try:
        data = api2.get_data_super(fitness = submit_fitness, hidden = hidden, region = 'USA', date_start = date_start)
        # For to submit
        for i in range(len(data['results'])):
            if (data['results'][i]['dateCreated'] <= last_update):
                api2.save(data['results'][i]['id'], 'hide', data['results'][i]['favorite'])
                print(data['results'][i]['id'])
                continue
            if author == data['results'][i]['author']:
                alpha_id = data['results'][i]['id']
                text = data['results'][i]['selection']['code']
                favorite = data['results'][i]['favorite']
                fitness = data['results'][i]['is']['fitness']
                if fitness >= float(submit_fitness):                
                    CHECK = 'PASS'
                    for qq in range(len(data['results'][i]['is']['checks'])):
                        if data['results'][i]['is']['checks'][qq]['result'] == 'FAIL':
                            print(data['results'][i]['is']['checks'][qq]['name'])
                            CHECK = 'FAIL'
                    if CHECK == 'PASS':
                        prod_corr = api2.get_prod_corr(alpha_id)
                        if prod_corr <= submit_prod_corr:
                            self_corr = api2.get_self_corr(alpha_id)
                            if self_corr <= submit_self_corr:
                                print('Code: ', text[0:200])     
                                print('Prod Corr: ', prod_corr)
                                print('Sharpe: ', data['results'][i]['is']['sharpe'])
                                fitness = data['results'][i]['is']['fitness']
                                print('Fitness:', fitness)
                                combo_des = random.choice(combo_dess)
                                k = api2.submit_super(alpha_id, k, favorite, combo_des, text)
                                print(k, "prod_corr == ", prod_corr, "self_corr == ", self_corr)
                                if k >= max_submit:
                                    sys.exit("Done")
                            else:
                                api2.save(alpha_id, 'hide', favorite)
                                print('alpha_id ===',alpha_id, ' Hide Corr Self === ', self_corr, " Corr All === ", prod_corr)
                        else:
                            api2.save(alpha_id, 'hide', favorite)
                            print('Hide Corr All', prod_corr)
                    else:
                        api2.save(alpha_id, 'hide', favorite)
                        print('Hide > 400')
                else:
                    api2.save(alpha_id, 'hide', favorite)
                    print('Hide > 400')
            else:
                print('Notice ...........')
                print('=========================')
                print(author)

                api2.save(alpha_id, 'hide', favorite)
    except Exception as e:
        print(e)
        pass
